# Le jeu Palworld et analyse des données

## Présentation du jeu Palworld

Palworld est un jeu de survie en monde ouvert développé par Pocket Pair, sorti en janvier 2024. Ce jeu combine des éléments de survie, de crafting, de collection de créatures et de construction de base. Il se déroule dans un monde ouvert peuplé de créatures appelées "Pals" que les joueurs peuvent capturer, élever et utiliser pour diverses tâches.

Les aspects clés du jeu incluent:

| Aspect |
| - |
| La capture et la collection de Pals (similaire à Pokémon) |
| L'utilisation des Pals pour combattre |
| L'exploitation des Pals pour des tâches comme la construction, l'agriculture, et le travail en usine |
| La construction et le développement d'un campement |
| L'exploration d'un monde ouvert avec différentes zones et biomes |

Le jeu a connu un immense succès dès sa sortie en raison de son concept innovant mêlant des mécaniques de jeux populaires comme Pokémon et Minecraft, mais avec une approche plus mature et parfois controversée.


## 📁 Analyse des Fichiers CSV et leur Rôle dans le Jeu

### Vue d'ensemble des Données Palworld

Sur la base des descriptions fournies, voici une analyse de chaque fichier de données et son importance dans le jeu :

### 1.  Palworld_Data--Palu combat attribute table.csv

**Contenu :**
- Statistiques de base (santé, attaque, défense, vitesse)
- Types élémentaires (Feu, Eau, Électricité, etc.)
- Compétences de combat spécifiques

**Rôle dans le jeu :** Ces données déterminent l'efficacité d'un Pal au combat. Les joueurs utilisent ces informations pour construire des équipes équilibrées et stratégiques pour les combats contre d'autres Pals ou les boss.

### 2.  Palworld_Data--Palu refresh level.csv

**Contenu :** Informations sur les niveaux auxquels les Pals apparaissent dans différentes zones du jeu.

**Rôle dans le jeu :** Cette information est cruciale pour les joueurs qui cherchent à capturer des Pals spécifiques. Elle permet de savoir où trouver certains Pals et à quel niveau ils apparaîtront, ce qui affecte leur difficulté de capture et leurs statistiques initiales.

### 3. Palworld_Data-Palu Job Skills Table.csv

**Contenu :** Compétences de travail que possèdent les différents Pals :
- Minage
- Bûcheronnage
- Agriculture
- Construction
- Cuisine
- Etc.

**Rôle dans le jeu :** Ces compétences déterminent quelles tâches un Pal peut effectuer dans votre campement. Un Pal avec une compétence de minage élevée sera efficace pour extraire des minerais, tandis qu'un Pal avec des compétences agricoles sera utile pour cultiver des ressources alimentaires.

### 4.  Palworld_Data-Tower BOSS attribute comparison.csv

**Contenu :** Attributs des boss spéciaux qui apparaissent dans les tours du jeu.

**Rôle dans le jeu :** Les tours sont des défis endgame où les joueurs affrontent des boss puissants. Ces données aident à comprendre la puissance relative de ces boss et à préparer des stratégies adaptées pour les combattre.

### 5.  Palworld_Data-comparison of ordinary BOSS attributes.csv

**Contenu :** Attributs des boss ordinaires que l'on trouve dans le monde ouvert.

**Rôle dans le jeu :** Ces boss représentent des défis importants durant l'exploration et fournissent des récompenses précieuses. Comprendre leurs attributs permet aux joueurs de se préparer adéquatement à ces affrontements.

### 6.  Palworld_Data-hide pallu attributes.csv

**Contenu :** Informations sur des attributs cachés des Pals qui ne sont pas immédiatement visibles dans le jeu.

**Rôle dans le jeu :** Ces attributs cachés peuvent inclure des taux de croissance spécifiques, des affinités particulières, ou des capacités spéciales qui se débloquent dans certaines conditions. Ces informations sont précieuses pour les joueurs qui cherchent à optimiser leurs Pals.

---
## Implications pour l'Analyse

Cette structure de données nous permettra d'analyser :
- **Optimisation des équipes de combat** via les attributs de combat
- **Gestion efficace des ressources** via les compétences de travail
- **Stratégies de capture** via les zones d'apparition et probabilités
- **Progression du jeu** via les niveaux et raretés des Pals

## Import des bibliothèques Python de l'étude

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mariadb
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
import os
from pathlib import Path

## Chargement des données brutes

In [4]:
# Définition du chemin vers les fichiers fournis
data_path = "../data/"

# Vérification de l'existence du repertoire , if not utiliser le repertoire courrant
if not os.path.exists(data_path):
    data_path = "./"

# Affichage des fichiers dispo pour la vérif
files = [f for f in os.listdir(data_path) if f.endswith('.csv')] # ici on liste tous les fichiers ou dossiers + on itère chaque élément + Filtre pour garder que les fichier .csv
print("Fichiers dispo:")
for file in files: 
    print(f"-{file}")

# Chargement des données brutes & gestion d'erreurs grâce à pd.read_csv en définissant le chemin vers les fichiers.csv 
try:
    combat_attribute = pd.read_csv(f'{data_path}Palworld_Data--Palu combat attribute table.csv')
    refresh_area = pd.read_csv(f'{data_path}Palworld_Data--Palu refresh level.csv')
    ordinary_boss = pd.read_csv(f'{data_path}Palworld_Data-comparison of ordinary BOSS attributes.csv')
    tower_boss = pd.read_csv(f'{data_path}Palworld_Data-Tower BOSS attribute comparison.csv') 
    job_skill = pd.read_csv(f'{data_path}Palworld_Data-Palu Job Skills Table.csv')
    hidden_attribute = pd.read_csv(f'{data_path}Palworld_Data-hide pallu attributes.csv')
    print("\nDonnées brutes chargées")
except FileNotFoundError as e : 
  print(f"Erreur: {e}. Vérifier les noms des fichiers et le chemin.")

Fichiers dispo:
-Palworld_Data--Palu combat attribute table.csv
-Palworld_Data--Palu refresh level.csv
-Palworld_Data-comparison of ordinary BOSS attributes.csv
-Palworld_Data-hide pallu attributes.csv
-Palworld_Data-Palu Job Skills Table.csv
-Palworld_Data-Tower BOSS attribute comparison.csv

Données brutes chargées


## Analyse exploratoires des données 

In [6]:
# Informations de base du DataFrame
def explore_dataframe(df, name):
    print(f"\n===== Exploration de {name} =====")
    print(f"Dimensions: {df.shape[0]} lignes, {df.shape[1]} colonnes")
    
    # Aperçu des premières lignes pour comprendre la structure
    print("\nAperçu des premières lignes:")
    display(df.head(3))
    
    # Types de données pour identifier colonnes numériques vs catégorielles
    print("\nTypes de données:")
    display(df.dtypes)
    
    # Statistiques descriptives complètes (moyenne, médiane, quartiles, etc.)
    print("\nStatistiques descriptives:")
    display(df.describe(include='all').T)
    
    # Détection et évaluations des valeurs manquantes
    print("\nValeurs manquantes:")
    missing = df.isnull().sum()
    missing_percent = (df.isnull().sum() / len(df)) * 100
    missing_data = pd.concat([missing, missing_percent], axis=1, keys=['Total', 'Pourcentage'])
    display(missing_data[missing_data['Total'] > 0])
    if missing_data['Total'].sum() == 0:
        print("Aucune valeur manquante détectée")
    
    # Détection des lignes doublons
    duplicates = df.duplicated().sum()
    print(f"\nNombre de lignes dupliquées: {duplicates}")
    
    # Analyse des colonnes catégorielles "objets"
    print("\nAnalyse des colonnes catégorielles:")
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    if len(categorical_cols) > 0:
        for col in categorical_cols:  # Possibilité de limiter le nombre de colonnes
            unique_values = df[col].unique()
            n_unique = len(unique_values)
            
            print(f"\n{col}:")
            print(f"  - {n_unique} valeurs uniques")
            
            # Affichage intelligent selon le nombre de valeurs uniques
            if n_unique <= 10:
                # Si peu de valeurs, les afficher toutes
                print(f"  - Valeurs: {unique_values}")
            elif n_unique <= 20:
                # Si modéré, afficher les plus fréquentes
                value_counts = df[col].value_counts().head(10)
                print(f"  - Top 10 des valeurs les plus fréquentes:")
                for val, count in value_counts.items():
                    print(f"    • {val}: {count} occurrences")
            else:
                # Si beaucoup de valeurs, donner juste les statistiques
                value_counts = df[col].value_counts()
                print(f"  - Top 5 des valeurs les plus fréquentes:")
                for val, count in value_counts.head(5).items():
                    print(f"    • {val}: {count} occurrences")
                print(f"  - Valeur la moins fréquente: {value_counts.tail(1).index[0]} ({value_counts.tail(1).values[0]} occurrence(s))")
    else:
        print("Aucune colonne catégorielle détectée")
    
    # Séparateur pour la lisibilité
    print("\n" + "="*70)

# Dictionnaire des datasets
datasets = {
   'Combat Attributes': combat_attribute,        
   'Job Skills': job_skill,                     
   'Hidden Attributes': hidden_attribute,        
   'Refresh Areas': refresh_area,               
   'Ordinary Boss Attributes': ordinary_boss,    
   'Tower Boss Attributes': tower_boss          
}


In [7]:
print(datasets)

{'Combat Attributes':     Table of Contents Palu Job Skill Table Skill Learning Level Tower BOSS Attributes Comparison with Normal BOSS Attributes Comparison of Partner Skill Values Individual Value Calculator  \
0                                                   ID                                                                                                                                        
1                                                    1                                                                                                                                        
2                                                    2                                                                                                                                        
3                                                    3                                                                                                                                        
4                      